## 02-multi layer logistic regression

[1.Load Dataset](#1)

[2.Build Parts of NN](#2)
  - [2-1.forward propagation](#2-1)  
  - [2-2.Compute Cost](#2-2)  
  - [2-3.Back Propagation](#2-3)  
  - [2-4.Update Parameters](#2-4)  

[3.Build Integrated NN](#3)

[4.Predict](#4)

In [1]:
import copy
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

<a id="1"></a>
## 1. Load Dataset

In [2]:
X, Y = make_moons(n_samples=5000, noise=0.2, random_state=42)
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.2, random_state=42)

In [3]:
train_y = train_y.reshape(-1, 1)
test_y = test_y.reshape(-1, 1)

train_x = train_x.T
train_y = train_y.T
test_x = test_x.T
test_y = test_y.T

print("train_x shape:", train_x.shape)
print("train_y shape:", train_y.shape)
print("test_x shape:", test_x.shape)
print("test_y shape:", test_y.shape)

train_x shape: (2, 4000)
train_y shape: (1, 4000)
test_x shape: (2, 1000)
test_y shape: (1, 1000)


<a id="2"></a>
## 2. Build Parts of NN

In [4]:
def layer_sizes(X, Y):
    n_x = X.shape[0]
    n_h = 4
    n_y = Y.shape[0]

    return n_x, n_h, n_y


def initialize_parameters(n_x, n_h, n_y):
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))

    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))

    parameters = {"W1":W1, "b1":b1, "W2":W2, "b2":b2}

    return parameters


def sigmoid(x):
    s = 1 / (1 + np.exp(-x))

    return s

In [5]:
n_x, n_h, n_y = layer_sizes(train_x, train_y)
parameters = initialize_parameters(n_x, n_h, n_y)

print(parameters["W1"].shape, parameters["b1"].shape)
print(parameters["W2"].shape, parameters["b2"].shape)

(4, 2) (4, 1)
(1, 4) (1, 1)


<a id="2-1"></a>
### 2-1. Forwad Propagation

$$z^{[1](i)} = W^{[1]} x^{(i)} + b^{[1]} \tag{1}$$ 
$$a^{[1](i)} = \tanh(z^{[1](i)}) \tag{2}$$
$$z^{[2](i)} = W^{[2]} a^{[1](i)} + b^{[2]} \tag{3}$$
$$\hat{y}^{(i)} = a^{[2](i)} = \sigma(z^{[2](i)}) \tag{4}$$
$$y^{(i)}_{\text{prediction}} = \begin{cases} 1 & \text{if } a^{[2](i)} > 0.5 \\ 0 & \text{otherwise} \end{cases} \tag{5}$$

In [6]:
def forward_propagation(X, parameters):
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]

    Z1 = np.matmul(W1, X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.matmul(W2, A1) + b2
    A2 = sigmoid(Z2)

    assert(A2.shape == (1, X.shape[1]))

    cache = {"Z1" : Z1, "A1" : A1, "Z2" : Z2, "A2" : A2}

    return A2, cache

In [7]:
A2, cache = forward_propagation(train_x, parameters)
print(A2.shape)

(1, 4000)


<a id="2-2"></a>
### 2-2. Compute Cost

$$J = - \frac{1}{m} \sum\limits_{i = 0}^{m} \large\left(\small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right)  \large  \right) \small \tag{6}$$

In [8]:
def compute_cost(A2, Y):
    m = Y.shape[1]
    loss = np.multiply(np.log(A2), Y) + np.multiply((1-Y), np.log(1-A2)) ## loss function
    cost = -np.sum(loss) / m ## cost function

    # cost = float(np.squeeze(cost))

    return cost

In [9]:
cost = compute_cost(A2, train_y)
print(cost)

0.6931103065067663


<a id="2-3"></a>
### 2-3. BackPropagation

1. 출력층의 오차:
   $$ \delta^{[2](i)} = \hat{y}^{(i)} - y^{(i)} $$

2. 출력층 가중치의 그래디언트:
   $$ \frac{\partial \mathcal{L}}{\partial W^{[2]}} = \delta^{[2](i)} a^{[1](i)T} $$

3. 출력층 편향의 그래디언트:
   $$ \frac{\partial \mathcal{L}}{\partial b^{[2]}} = \delta^{[2](i)} $$

4. 첫 번째 층의 오차:
   $$ \delta^{[1](i)} = (W^{[2]T} \delta^{[2](i)}) * (1 - (a^{[1](i)})^2) $$

5. 첫 번째 층 가중치의 그래디언트:
   $$ \frac{\partial \mathcal{L}}{\partial W^{[1]}} = \delta^{[1](i)} x^{(i)T} $$

6. 첫 번째 층 편향의 그래디언트:
   $$ \frac{\partial \mathcal{L}}{\partial b^{[1]}} = \delta^{[1](i)} $$


In [10]:
def backward_propagation(parameters, cache, X, Y):
    m = X.shape[1]

    W1 = parameters["W1"]
    W2 = parameters["W2"]
    A1 = cache["A1"]
    A2 = cache["A2"]

    dZ2 = A2 - Y

    dW2 = (1 / m) * np.dot(dZ2, A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)

    dZ1 = np.dot(W2.T, dZ2) * (1 - np.power(A1, 2))
    dW1 = (1 / m) * np.dot(dZ1, X.T)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)

    grads = {"dW1" : dW1, "db1" : db1, "dW2" : dW2, "db2" : db2}

    return grads

In [11]:
grads = backward_propagation(parameters, cache, train_x, train_y)

print(grads["dW1"].shape)
print(grads["db1"].shape)
print(grads["dW2"].shape)
print(grads["db2"].shape)

(4, 2)
(4, 1)
(1, 4)
(1, 1)


<a id="2-4"></a>
### 2-4. Update Parameters

In [12]:
def update_parameters(parameters, grads, learning_rate=1.2):
    W1 = copy.deepcopy(parameters["W1"])
    b1 = parameters["b1"]
    W2 = copy.deepcopy(parameters["W2"])
    b2 = parameters["b2"]

    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]

    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2

    parameters = {"W1" : W1, "b1" : b1, "W2" : W2, "b2" : b2}

    return parameters

In [13]:
parameters = update_parameters(parameters, grads)
print(parameters["W1"].shape)
print(parameters["b1"].shape)
print(parameters["W2"].shape)
print(parameters["b2"].shape)

(4, 2)
(4, 1)
(1, 4)
(1, 1)


<a id="3"></a>
## 3. Build Integrated NN

In [14]:
def nn_model(X, Y, n_h, num_iterations = 1000, print_cost=True):
    np.random.seed(3)
    n_x, n_h, n_y,  = layer_sizes(X, Y)

    parameters = initialize_parameters(n_x, n_h, n_y)
    for i in range(0, num_iterations):
        A2, cache = forward_propagation(X, parameters)
        cost = compute_cost(A2, Y)

        grads = backward_propagation(parameters, cache, X, Y)
        parameters = update_parameters(parameters, grads)

        if print_cost and i % 100 == 0:
            print(f"Cost after iteration {i} : {cost}")

    return parameters

In [15]:
parameters = nn_model(train_x, train_y, 4)

Cost after iteration 0 : 0.6931580212201081
Cost after iteration 100 : 0.2937521294520586
Cost after iteration 200 : 0.2927814873086575
Cost after iteration 300 : 0.29219237636363016
Cost after iteration 400 : 0.29179809019473474
Cost after iteration 500 : 0.2915138127613999
Cost after iteration 600 : 0.29129755744484653
Cost after iteration 700 : 0.2911264648923138
Cost after iteration 800 : 0.29297486079499957
Cost after iteration 900 : 0.2929205654417771


<a id="4"></a>
## 4.Predict

In [16]:
def predict(parameters, X):
    A2, cache = forward_propagation(X, parameters)
    predictions = (A2 >= 0.5).astype(np.int32)

    return predictions

In [17]:
predictions = predict(parameters, train_x)
accuracy = float((np.dot(train_y, predictions.T) + np.dot(1 - train_y, 1 - predictions.T)) / float(train_y.size) * 100)
print (f'Accuracy: {accuracy}%')

Accuracy: 86.375%


/tmp/ipykernel_888923/2236656979.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  accuracy = float((np.dot(train_y, predictions.T) + np.dot(1 - train_y, 1 - predictions.T)) / float(train_y.size) * 100)
